In [8]:
# hw0 p1 sample way complete
def parse_polynomial(poly_str):
    terms = []
    poly_str = poly_str.replace(" ", "")  # Remove any spaces
    poly_str = poly_str.replace("-", "+-")  # Handle negative terms by replacing '-' with '+-'

    # Split the polynomial into terms using '+' as the delimiter
    split_terms = poly_str.split("+")
    
    for term in split_terms:
        if term:  # Skip any empty terms (which may occur due to leading '+')
            terms.append(parse_term(term))
    
    return terms

def parse_term(term_str):
    # Default to a constant if no variables
    coeff = 1
    exponents = {}  # Dictionary to store variables and their exponents

    # Handle negative sign
    if term_str.startswith('-'):
        sign = -1
        term_str = term_str[1:]
    else:
        sign = 1

    # Separate coefficient and variable parts
    coeff_str = ""
    var_part = ""

    i = 0
    # Extract the coefficient part if present (numbers before the first alphabet)
    while i < len(term_str) and term_str[i].isdigit():
        coeff_str += term_str[i]
        i += 1

    # Extract the variable part after the coefficient
    var_part = term_str[i:]

    # Set the coefficient
    if coeff_str:
        coeff = int(coeff_str) * sign
    else:
        coeff = sign  # If no coefficient is provided, assume it's 1 or -1

    # Now parse the variable part
    i = 0
    while i < len(var_part):
        if var_part[i].isalpha():  # Detect variable name
            var_name = var_part[i]
            i += 1
            # Check if there's an exponent
            if i < len(var_part) and var_part[i] == '^':
                i += 1  # Move past '^'
                exp_str = ""
                while i < len(var_part) and var_part[i].isdigit():  # Extract the exponent
                    exp_str += var_part[i]
                    i += 1
                exponent = int(exp_str)
            else:
                exponent = 1  # If no exponent is given, assume it's 1
            # Store the variable and its exponent
            exponents[var_name] = exponent
        else:
            i += 1  # Move to the next character

    return (exponents, coeff)

def multiply_multivariable_polynomials(p1, p2):
    result = []
    
    # Multiply each term from p1 with each term from p2
    for exp1, coeff1 in p1:
        for exp2, coeff2 in p2:
            new_exp = exp1.copy()  # Copy the first term's exponents
            for var, exp in exp2.items():
                if var in new_exp:
                    new_exp[var] += exp  # Add exponents if variable already exists
                else:
                    new_exp[var] = exp  # Add new variable with its exponent
            result.append((new_exp, coeff1 * coeff2))
    
    result = combine_like_terms(result)
    return result

def combine_like_terms(poly):
    combined = {}
    
    # Combine terms with the same exponents
    for exp, coeff in poly:
        # Convert exponents dict to a tuple for use as a key
        exp_tuple = tuple(sorted(exp.items()))
        if exp_tuple in combined:
            combined[exp_tuple] += coeff
        else:
            combined[exp_tuple] = coeff
    
    # Convert back to list format
    result = [(dict(exp), coeff) for exp, coeff in combined.items()]
    return result

def format_polynomial(poly):
    terms = []
    
    for exp, coeff in poly:
        # If the coefficient is zero, skip this term
        if coeff == 0:
            continue
        
        # Format the coefficient and variables
        term = ""
        if coeff > 0 and len(terms) > 0:  # Add '+' sign if it's a positive term and not the first term
            term += "+"
        
        # If the coefficient is not 1 or -1, include it with a '*' for variable terms
        if abs(coeff) != 1 or not exp:
            term += str(coeff)
        elif coeff == -1:
            term += "-"

        # Add '*' if there are variables and the coefficient is not zero
        if exp and abs(coeff) != 1:
            term += "*"

        # Add the variable part
        for var, power in exp.items():
            if power == 1:
                term += var
            else:
                term += f"{var}^{power}"
            term += "*"
        
        # Remove trailing '*' if present
        if term.endswith("*"):
            term = term[:-1]
        
        terms.append(term)
    
    return "".join(terms)

def parse_polynomial_expression(expression):
    # Find and split the polynomials by splitting at each pair of parentheses
    poly_strings = expression.split(")(")
    
    # Clean up any remaining parentheses from the individual polynomials
    poly_strings = [poly_str.replace("(", "").replace(")", "") for poly_str in poly_strings]
    
    # Parse each polynomial
    polynomials = [parse_polynomial(poly_str) for poly_str in poly_strings]
    
    return polynomials

def multiply_multiple_polynomials(polynomials):
    # Start with the first polynomial
    result = polynomials[0]
    
    # Multiply with each subsequent polynomial
    for poly in polynomials[1:]:
        result = multiply_multivariable_polynomials(result, poly)
    
    return result

# Example usage
# expression = "(A+2*B^2)(B+3*C^3)(2*A+B+C)"
expression = input()
# Parse the expression into multiple polynomials
polynomials = parse_polynomial_expression(expression)

# Multiply all the polynomials together
result = multiply_multiple_polynomials(polynomials)

# Format and output the result
formatted_result = format_polynomial(result)
print("The result of the polynomial multiplication is:", formatted_result)


The result of the polynomial multiplication is: 2*A^2*B+A*B^2+A*B*C+6*A^2*C^3+3*A*B*C^3+3*A*C^4+4*A*B^3+2*B^4+2*B^3*C+12*A*B^2*C^3+6*B^3*C^3+6*B^2*C^4
